In [3]:
import pandas as pd

file_path_lokal = r'D:\Sem 7\Sistem Rekomendasi\proyek-sistem-informasi\data-books\train.csv'
ratings_df = pd.read_csv(file_path_lokal)

print("Data berhasil dimuat!")
ratings_df.head()


Data berhasil dimuat!


,user_id,item_id
0,8,0002005018
1,8,074322678X
2,8,0887841740
3,8,1552041778
4,8,1567407781


In [4]:
ratings_df.columns


Index(['user_id', 'item_id'], dtype='object')

In [10]:
ratings_df['rating'] = 1.0

In [11]:
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

ratings_df['user_id_enc'] = user_encoder.fit_transform(ratings_df['user_id'])
ratings_df['item_id_enc'] = item_encoder.fit_transform(ratings_df['item_id'])


In [12]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    ratings_df,
    test_size=0.2,
    random_state=42
)

print("Train:", train_df.shape)
print("Test :", test_df.shape)


Train: (215811, 5)
Test : (53953, 5)


In [13]:
from scipy.sparse import csr_matrix

train_matrix = csr_matrix(
    (train_df['rating'], (train_df['user_id_enc'], train_df['item_id_enc']))
)

test_matrix = csr_matrix(
    (test_df['rating'], (test_df['user_id_enc'], test_df['item_id_enc']))
)

print("Sparse matrix berhasil dibuat!")
print("Train matrix shape:", train_matrix.shape)
print("Test matrix shape :", test_matrix.shape)

Sparse matrix berhasil dibuat!
Train matrix shape: (13876, 123069)
Test matrix shape : (13876, 123058)


In [16]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from implicit.nearest_neighbours import ItemItemRecommender
from implicit.evaluation import train_test_split, mean_average_precision_at_k

print("Semua library berhasil di-import.")
file_path_lokal = r'D:\Sem 7\Sistem Rekomendasi\proyek-sistem-informasi\data-books\train.csv'
file_path_lokal = file_path_lokal.strip().strip('"')

try:
    df = pd.read_csv(file_path_lokal)
    print(f"\nData '{file_path_lokal}' berhasil dimuat.")
    
    df['user_id_cat'] = df['user_id'].astype("category")
    df['item_id_cat'] = df['item_id'].astype("category")
    
    n_users = len(df['user_id_cat'].cat.categories)
    n_items = len(df['item_id_cat'].cat.categories)
    # Pastikan angka ini sesuai dengan error Anda:
    print(f"Users: {n_users}, Items: {n_items}") 

    # ===== INI ADALAH PERBAIKAN KRUSIAL =====
    # Pastikan (baris, kolom) adalah (user, item)
    interactions_matrix = sparse.csr_matrix(
        (np.ones(df.shape[0]),
         (df['user_id_cat'].cat.codes,  # <-- Baris = USER
          df['item_id_cat'].cat.codes) # <-- Kolom = ITEM
        ),
        shape=(n_users, n_items) # <-- Shape = (Users, Items)
    )
    # =======================================
    
    print(f"Shape Matriks: {interactions_matrix.shape}") # <-- HARUS (Users, Items)
    
    train_matrix, test_matrix = train_test_split(interactions_matrix, train_percentage=0.8, random_state=42)
    print(f"Shape Train Matrix: {train_matrix.shape}") # <-- HARUS (Users, Items)

    print("\nMulai melatih model baseline (IBCF)...")
    model_baseline = ItemItemRecommender(K=20)
    model_baseline.fit(train_matrix) # <-- Benar: (Users, Items)
    print("Model baseline (IBCF) selesai dilatih.")

    print("\nMulai menghitung skor MAP@10 untuk model baseline...")
    map10_baseline = mean_average_precision_at_k(model_baseline,
                                                 train_matrix, # <-- Benar: (Users, Items)
                                                 test_matrix,  # <-- Benar: (Users, Items)
                                                 K=10,
                                                 num_threads=0) 

    print("\n--- HASIL MODEL BASELINE (IBCF) ---")
    print(f"Skor MAP@10: {map10_baseline}")

except Exception as e:
    print(f"\nTerjadi error di Sel 1: {e}")

Semua library berhasil di-import.

Data 'D:\Sem 7\Sistem Rekomendasi\proyek-sistem-informasi\data-books\train.csv' berhasil dimuat.
Users: 13876, Items: 123069
Shape Matriks: (13876, 123069)
Shape Train Matrix: (13876, 123069)

Mulai melatih model baseline (IBCF)...


  0%|          | 0/123069 [00:00<?, ?it/s]

Model baseline (IBCF) selesai dilatih.

Mulai menghitung skor MAP@10 untuk model baseline...


  0%|          | 0/11454 [00:00<?, ?it/s]


--- HASIL MODEL BASELINE (IBCF) ---
Skor MAP@10: 0.018602178412511838


In [17]:
from implicit.als import AlternatingLeastSquares
import time
from threadpoolctl import threadpool_limits 
from implicit.evaluation import mean_average_precision_at_k



print("Library untuk model ALS (AlternatingLeastSquares) berhasil di-import.")

# --- 1. Inisialisasi Model Canggih (ALS) ---

# Rekomendasi perbaikan untuk kode Anda (library implicit)
model_als = AlternatingLeastSquares(factors=256, 
                                  regularization=0.1, 
                                  iterations=25, 
                                  alpha=15.0, 
                                  random_state=42)
print("Model ALS berhasil diinisialisasi.")

# --- 2. Latih Model ALS ---

print("\nMulai melatih model canggih (ALS)...")
start_time = time.time()

# Mengatasi peringatan MKL (opsional tapi disarankan)
# Ini akan mematikan 'internal threadpool' MKL agar tidak bentrok
threadpool_limits(1, "blas")

# latih model HANYA pada data latih (train_matrix)

model_als.fit(train_matrix.T.tocsr()) 
end_time = time.time()
print(f"Model ALS selesai dilatih. Waktu: {end_time - start_time:.2f} detik")

# --- 3. Evaluasi Model ALS (MAP@10) ---
print("\nMulai menghitung skor MAP@10 untuk model ALS...")

# Hitung skor MAP@10
map10_als = mean_average_precision_at_k(model_als,

                                        train_matrix.T.tocsr(),

                                        test_matrix.T.tocsr(),

                                        K=10,

                                        num_threads=0) # Menggunakan semua core
print("\n--- HASIL MODEL CANGGIH (ALS) ---")
print(f"Skor MAP@10: {map10_als}")
print("\n--- PERBANDINGAN ---")

# Ambil 'map10_baseline' dari variabel yang sudah Anda jalankan di sel sebelumnya

print(f"Baseline (IBCF) MAP@10: {map10_baseline}") 
print(f"Advanced (ALS) MAP@10 : {map10_als}")

if map10_als > map10_baseline:

    print("\nSELAMAT! Model ALS Anda berhasil mengalahkan baseline. 👍")
else:

    print("\nModel ALS belum mengalahkan baseline. Coba 'tuning' parameter (factors, regularization, iterations).")

Library untuk model ALS (AlternatingLeastSquares) berhasil di-import.
Model ALS berhasil diinisialisasi.

Mulai melatih model canggih (ALS)...


  0%|          | 0/25 [00:00<?, ?it/s]

Model ALS selesai dilatih. Waktu: 38.29 detik

Mulai menghitung skor MAP@10 untuk model ALS...


  0%|          | 0/36233 [00:00<?, ?it/s]


--- HASIL MODEL CANGGIH (ALS) ---
Skor MAP@10: 0.023412103077860163

--- PERBANDINGAN ---
Baseline (IBCF) MAP@10: 0.018602178412511838
Advanced (ALS) MAP@10 : 0.023412103077860163

SELAMAT! Model ALS Anda berhasil mengalahkan baseline. 👍
